In [3]:
import pandas as pd
import numpy as np
# import nltk
# nltk.download('punkt')
from collections import Counter
# import fasttext.util
import os
import re
from collections import Counter
from tqdm.auto import tqdm
tqdm.pandas()
# fasttext.util.download_model('en', if_exists='ignore')

# HuggingFace

In [7]:
# ! pip install transformers

In [8]:
#  ! pip install datasets

In [11]:
import pandas as pd
import numpy as np

import torch

from transformers import AutoTokenizer
from datasets import load_dataset

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [10]:
torch.cuda.is_available()

False

## Preparing the data

In [25]:
# doc_df = pd.read_csv('data/Thoughts_en.doc.tsv', sep='\t')
doc_df = pd.read_csv('data/subs.actions.tsv', sep='\t')

In [26]:
doc_df = doc_df.dropna()

In [27]:
msk = np.random.rand(len(doc_df)) < .95

In [28]:
train_doc_df = doc_df[msk]

In [29]:
test_doc_df = doc_df[~msk]

In [30]:
# train_doc_df.to_csv('data/thoughts/Thoughts_en.doc.train.tsv', header=['text','label'], index=False, sep='\t')
train_doc_df.to_csv('data/subs.actions.train.tsv', header=['text','label'], index=False, sep='\t')

In [31]:
# test_doc_df.to_csv('data/thoughts/Thoughts_en.doc.test.tsv', header=['text','label'], index=False, sep='\t')
test_doc_df.to_csv('data/subs.actions.test.tsv', header=['text','label'], index=False, sep='\t')

In [45]:
# vec_df = pd.read_csv('data/thoughts/Thoughts_en.vec.tsv', sep='\t', header=None)

In [46]:
# train_vec_df = vec_df[msk]

In [203]:
# test_vec_df = vec_df[~msk]

In [209]:
# train_vec_df.to_csv('data/Thoughts_en.vec.train.tsv', index=False, header=False, sep='\t')

In [210]:
# test_vec_df.to_csv('data/Thoughts_en.vec.test.tsv', index=False, header=False, sep='\t')

In [222]:
# train_vec_df['label'] = train_doc_df['is_action']

In [223]:
# test_vec_df['label'] = test_doc_df['is_action']

## Training

In [224]:
# train_vec_df.to_csv('data/Thoughts_en.vec.train.labeled.tsv', index=False, sep='\t')
# test_vec_df.to_csv('data/Thoughts_en.vec.test.labeled.tsv', index=False, sep='\t')

In [32]:
# dataset = load_dataset('csv', data_files={'train': 'data/thoughts/Thoughts_en.doc.train.tsv',
#                                               'test': 'data/thoughts/Thoughts_en.doc.test.tsv'}, delimiter='\t')
dataset = load_dataset('csv', data_files={'train': 'data/subs.actions.train.tsv',
                                              'test': 'data/subs.actions.test.tsv'}, delimiter='\t')

Using custom data configuration default-abee3eae27b5665e


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-abee3eae27b5665e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [63]:
# dataset = load_dataset("imdb")

In [33]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 176727
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9305
    })
})

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [35]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=20)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/177 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [36]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [37]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [41]:
training_args = TrainingArguments("subs_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
# trainer = Trainer(
#     model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
# )
trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [13]:
trainer.train()

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("subs_trainer/checkpoint-66000/")

In [65]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt")
    labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
    outputs = model(**inputs, labels=labels)
    return torch.nn.Softmax(dim=1)(outputs.logits).tolist()[0][1]
    

In [66]:
predict("Alice, you need to finish my cereal by tomorrow")
# inputs = tokenizer("Fuck you", return_tensors="pt")



0.9999179840087891

In [67]:
subs_df = pd.read_csv('data/subs.actions.test.tsv', sep='\t')

In [69]:
subs_df = subs_df[subs_df['label']==False]

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [71]:
subs_df['prediction'] = subs_df['text'].progress_apply(predict)

  0%|          | 0/4636 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
subs_df = subs_df.sort_values('prediction', ascending=False)

In [76]:
subs_df.iloc[1,0]

'Downloading the data would cause death to the data carrier.'

## Use POS to identify Action Items

In [14]:
# ! pip install nltk

In [15]:
# ! python -m spacy download en

In [16]:
import spacy
from string import punctuation
import nltk
import nltk.data
from collections import Counter

In [16]:
# nltk.download()

In [17]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [18]:
nlp = spacy.load("en_core_web_sm")

In [19]:
doc_df = pd.read_csv('data/thoughts/Thoughts_en.doc.tsv', sep='\t')

In [20]:
doc_df = pd.read_csv('data/subs.actions.tsv', sep='\t')

In [21]:
subs_df = doc_df[doc_df['label']==False]

In [24]:
verbs = []
def is_action(text):
    if type(text)!=str:
        return False
    if "?" in text:
        return False
#     if 'need' in text:
#         return True
    text = re.sub(',', '.', text)
#     split = tokenizer.tokenize(text)
    sents = nlp(text)
    for sent in sents.sents:
        if not sent:
            continue
        for word in sent:
            if word.dep_=='neg':
                return False
            if word.dep_=='ROOT' and word.pos_=='VERB':
                try:
                    morph = word.morph.to_dict()
                    if morph['VerbForm']=='Inf':
                        return True
                except KeyError:
                    continue
#         if sen[0].pos_=='VERB':
#             return True
#     #     if '!' in x and any(word.pos_=='VERB' for word in sen): # Doesn't work
#     #         return True
    return False

In [ ]:
# doc_df['is_action'] = doc_df['doc'].progress_apply(is_action)
subs_df['is_action'] = subs_df['text'].progress_apply(is_action)

  0%|          | 0/93016 [00:00<?, ?it/s]

In [26]:
len(verbs)

0

In [139]:
cnt = Counter([verb.text.lower() for verb in verbs ])

In [140]:
cnt.most_common(20)

[('have', 102),
 ('know', 90),
 ('go', 80),
 ('get', 74),
 ('got', 70),
 ('come', 67),
 ('let', 66),
 ('do', 54),
 ('see', 47),
 ('going', 43),
 ('need', 42),
 ('gon', 40),
 ('want', 37),
 ('look', 36),
 ('think', 35),
 ('tell', 34),
 ('say', 31),
 ('thank', 30),
 ('take', 29),
 ('doing', 28)]

In [171]:
sents = nlp(subs_df.loc[19, 'text'])

In [175]:
for word in sents:
    print(word, word.dep_)

They nsubj
wo aux
n't neg
waste ROOT
any det
time dobj
. punct


In [70]:
actions_df = subs_df[ subs_df['is_action'] & subs_df['names'] & subs_df['dates']]

In [67]:
def get_names(sent):
    sentence = nlp(sent)
    names = []
    for token in sentence:
        if token.pos_=='PROPN':
            names.append(token)
    return names

In [45]:
subs_df['names'] = subs_df['text'].progress_apply(get_names)

  0%|          | 0/93016 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
sent = nlp("I will be there April 31")

In [61]:
def get_dates(sent):
    sentence = nlp(sent)
    dates = []
    for entity in sentence.ents:
        if entity.label_=="DATE":
            dates.append(entity)
    return dates
    

In [ ]:
subs_df['dates'] = subs_df['text'].progress_apply(get_dates)

  0%|          | 0/93016 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
doc_df.to_csv('data/Thoughts_en.doc.tsv', index=False, sep='\t')

In [63]:
subs_df.to_csv('data/subs.action.name.date.doc.tsv', index=False, sep='\t')

In [72]:
actions_df.to_csv('data/subs.action.name.date.doc.tsv', index=False, sep='\t')

# Generate synthetic examples

In [3]:
import random

In [4]:
# time
# time template
preposition = ["by","for","until", 'till', '']
timeframe = ["end of", "next", "this", ""]
day = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "today", "tomorrow", "week", "the 20th", "the first of July", ""]
time = ["3pm", "afternoon", "morning", "at 5pm", "noon", ""]

def gen_time():
    prep = random.choice(preposition)
    tf = random.choice(timeframe)
    d = random.choice(day)
    t = random.choice(time)

    return " ".join([prep, tf, d, t])

# 
# name
# action


In [5]:
# ! pip install lxml

In [7]:
name_dfs = pd.read_html('https://www.ssa.gov/oact/babynames/decades/century.html')

In [8]:
name_df = name_dfs[0]

In [9]:
names = list(name_df['Males']["Name"])+list(name_df['Females']["Name"])

In [10]:
# action
asks = ["can you please", "can you", "please", "", "you need to"]
actions = ["read", "write", "fix", "go over", "correct", "submit", "discuss", "run", "execute", "improve", "purchase", "review"]
adjectives = ["the last", "the", "the new", "this", "that", "the old"]
subjects = ["report", "code", "account", "review", "data", "record", "description"]

def gen_action():
    a = random.choice(asks)
    act = random.choice(actions)
    adj = random.choice(adjectives)
    subj = random.choice(subjects)

    return " ".join([a, act, adj, subj])

In [11]:
templates = [
    "{name}, {action} {time}",
    "{name}, {time} {action}",
    "{action} {time}, {name}",
    "{time} {action}, {name}"
]

def gen_action_item():
    name = random.choice(names)
    action = gen_action()
    time = gen_time()
    temp = random.choice(templates)
    
    

    action_item = temp.format(name=name, action=action, time=time)
    return action_item

In [12]:
gen_action_item()

' end of Thursday morning you need to improve this data, Brandon'

# OpenSubtitles

In [13]:
open_sub = load_dataset('open_subtitles', 'en-hi')

Reusing dataset open_subtitles (/root/.cache/huggingface/datasets/open_subtitles/en-hi/2018.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
en_sub = [x['en'] for x in open_sub['train']['translation']]

In [16]:
action_items = [gen_action_item() for _ in range(len(en_sub))]

In [21]:
subs_df = pd.DataFrame(list(zip(en_sub, [False]*len(en_sub))) + list(zip(action_items,[True]*len(action_items))), columns=['text', 'label'])

In [22]:
subs_df.head()

,text,label
0,THE BICYCLE THIEF,False
1,Ricci?,False
2,Is Ricci there?,False
3,Are you deaf? Come on!,False
4,Get a move on.,False


In [23]:
subs_df.tail()

,text,label
186027,please submit this description next the 20th ...,True
186028,you need to write the old review by next Tuesd...,True
186029,"Patrick, till this the 20th noon can you submi...",True
186030,can you go over the new account till this Frid...,True
186031,"Donna, until afternoon please fix the last r...",True


In [24]:
subs_df.to_csv('data/subs.actions.tsv', index=False, sep='\t')

In [ ]:
def is_action(text):
    sen = nlp(text)
    
    

In [80]:
subs_df[subs_df['text'].apply(lambda x: '!' in x, sen = nlp(x))]

,text,label,prediction
2935,Shit!,False,0.00004
2954,Shit!,False,0.00004
2534,Shit!,False,0.00004
2740,Shit! Fuck!,False,0.00004
3443,And away!,False,0.00004
...,...,...,...
4137,The hero that you are I shall transform into a...,False,0.00004
1171,"I swear on Allah, the great!",False,0.00004
2196,Remove the containment lid!,False,0.00004
3874,"Enemies of justice, prepare for war!",False,0.00004
